In [0]:
from yt import wrapper as yt

You can use YTsaurus client without passing a proxy and a token.
YTsaurus client implicitly uses the token from the YT_TOKEN secret and works with the current cluster.

Let's get attributes from `//sys` directory.

In [3]:
yt.get("//sys/@")

{'id': '1-1f-1344025a-641e14df', 'type': 'sys_node', 'builtin': false, 'sequoia': false, 'ref_counter': 1, 'ephemeral_ref_counter': 0, 'weak_ref_counter': 0, 'foreign': false, 'native_cell_tag': 4932, 'revision': 52613349834784, 'attribute_revision': 52613349834784, 'content_revision': 52334176511585, 'inherit_acl': true, 'acl': [], 'owner': 'root', 'effective_acl': YsonEntity, 'user_attribute_keys': ['enable_safe_mode', 'disable_bundle_balancer', 'ui_config'], 'opaque_attribute_keys': YsonEntity, 'user_attributes': YsonEntity, 'life_stage': 'creation_committed', 'estimated_creation_time': YsonEntity, 'parent_id': 'ffffffff-ffffffff-1344012f-1d0e63f7', 'external': false, 'locks': YsonEntity, 'lock_count': 0, 'lock_mode': 'none', 'path': YsonEntity, 'key': 'sys', 'effective_expiration': YsonEntity, 'creation_time': '2023-09-14T08:58:43.575130Z', 'modification_time': '2025-01-16T16:46:20.862120Z', 'access_time': '2025-01-21T19:22:49.808213Z', 'access_counter': 170775, 'resource_usage': {

Create ytsaurus client, create table and sort it.

In [5]:
client = yt.YtClient(config=yt.default_config.get_config_from_env())

In [6]:
import uuid

base_example_path = f"//tmp/jupyter-example-{uuid.uuid4().hex}"
client.create("map_node", base_example_path)
print(base_example_path)

//tmp/jupyter-example-f5c1804f176a4b02b94f28ee9c016ac2


In [7]:
records = [{"key": "a", "value": 3}, {"key": "b", "value": 2}, {"key": "c", "value": 1}]
table_path = f"{base_example_path}/table_to_be_sorted"
client.write_table(f"{base_example_path}/table_to_be_sorted", input_stream=records, force_create=True)

In [8]:
client = yt.YtClient(config=yt.default_config.get_config_from_env())

In [9]:
print([r for r in client.read_table(table_path)])

[{'key': 'a', 'value': 3}, {'key': 'b', 'value': 2}, {'key': 'c', 'value': 1}]


In [10]:
client.run_sort(table_path, table_path, sort_by="value")

2025-01-21 19:28:04,904	INFO	Operation started: https://playground.yt.nebius.yt/playground/operations/c88c20f4-5ee51517-134403e8-734c929f/details


2025-01-21 19:28:04,944	INFO	( 0 min) operation c88c20f4-5ee51517-134403e8-734c929f initializing


2025-01-21 19:28:06,076	INFO	( 0 min) operation c88c20f4-5ee51517-134403e8-734c929f: running=0     completed=0     pending=1     failed=0     aborted=0     lost=0     total=1     blocked=0    


2025-01-21 19:28:08,882	INFO	( 0 min) operation c88c20f4-5ee51517-134403e8-734c929f: running=1     completed=0     pending=0     failed=0     aborted=0     lost=0     total=1     blocked=0    


2025-01-21 19:28:15,687	INFO	( 0 min) operation c88c20f4-5ee51517-134403e8-734c929f completing


2025-01-21 19:28:16,217	INFO	( 0 min) operation c88c20f4-5ee51517-134403e8-734c929f completed


In [11]:
print([r for r in client.read_table(table_path)])

[{'value': 1, 'key': 'c'}, {'value': 2, 'key': 'b'}, {'value': 3, 'key': 'a'}]
